#Mounting Google Drive for Data Access:

Mounting Google Drive within the Colab environment to establish a connection and access data files stored in the Google Drive directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Setting File Paths for Data Access:

 Defining the file paths to the label and image data directories for accessing data files within the project.

In [ ]:
label_file_path = r"/content/drive/MyDrive/CV_project/data/label/label.lst"
image_folder_path = r"/content/drive/MyDrive/CV_project/origin"

#Loading and Structuring Label Data:

Loading label data from a CSV file using pandas, defining column names for the DataFrame, and structuring the data into a tabular format for further analysis and processing.

In [ ]:
import pandas as pd
df_info = pd.read_csv(label_file_path, sep=' ', header=None)
col_names = "image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_cofidence expression_label".split(' ')
df_info.columns = col_names
df_info

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


#Iterating and Extracting Data from DataFrame:
 Iterating through the rows of the DataFrame to extract specific values, such as image names and face box confidences, and printing the extracted values.





In [ ]:
for ind in df_info.index:
  df_iterate = df_info['image_name'][ind], df_info['face_box_cofidence'][ind]

print(df_iterate)

('surprised_expression_394.jpg', 77.7758)


#Selecting High-Confidence Data Entries:

 Filtering the DataFrame to select data entries with a face box confidence score greater than 65, thereby focusing on high-confidence facial detections for further analysis or processing.

In [ ]:
df_sel = df_info[df_info.face_box_cofidence > 60]
df_sel

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
5,angry_actor_137.jpg,0,93,468,842,467,88.9519,0
9,angry_actor_150.jpg,0,56,263,376,169,81.8792,0
11,angry_actor_16.jpg,0,158,249,521,430,66.4535,0
...,...,...,...,...,...,...,...,...
91786,surprised_expression_409.jpg,0,80,46,184,218,63.8069,5
91787,expressionless_husband_673.jpg,0,48,194,388,242,82.2975,4
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5


#Iterating Through Selected Data Entries:

 Iterating through the selected DataFrame df_sel, extracting specific attributes such as image name and bounding box coordinates, and storing them for further processing. The final extracted row's information is displayed, which might not be consistent with the iteration due to overwriting.

In [ ]:
for i, row in df_sel.iterrows():
  image_name = row['image_name']
  x1 = row['face_box_left']
  x2 = row['face_box_right']
  y1 = row['face_box_top']
  y2 = row['face_box_bottom']
  a = [x1,x2, y1,y2]

print(row)

image_name            surprised_expression_394.jpg
face_id_in_image                                 0
face_box_top                                    47
face_box_left                                   38
face_box_right                                 152
face_box_bottom                                161
face_box_cofidence                         77.7758
expression_label                                 5
Name: 91792, dtype: object


In [ ]:
import os
import cv2
from tqdm import tqdm_gui

X = []
Y = []

image_directory = '/content/drive/MyDrive/CV_project/origin'
cropped_directory = '/content/drive/MyDrive/CV_project/data/image'

for i, row in df_sel.sample(5000).iterrows():
    image_name = row['image_name']
    x1 = row['face_box_left']
    x2 = row['face_box_right']
    y1 = row['face_box_top']
    y2 = row['face_box_bottom']
    label = row['expression_label']

    cropped_path = os.path.join(cropped_directory, f"cropped_{image_name}")

    # Read the cropped image
    comb_path = os.path.join(image_directory, image_name)
    cropped_image = cv2.imread(comb_path)

    if cropped_image is not None:
        face = cropped_image[x1:x2, y1:y2]

        # Debugging: Print face shape and cropping coordinates
        print(f"Face shape: {face.shape}, x1={x1}, x2={x2}, y1={y1}, y2={y2}")
    else:
        continue

    if face is not None and not face.size == 0:
      face_resized = cv2.resize(face, (64, 64))
    else:
        continue

    X.append(face_resized)
    Y.append(label)

Face shape: (95, 95, 3), x1=295, x2=390, y1=71, y2=166
Face shape: (48, 48, 3), x1=36, x2=84, y1=308, y2=356
Face shape: (114, 114, 3), x1=238, x2=352, y1=47, y2=161
Face shape: (134, 134, 3), x1=269, x2=403, y1=44, y2=178
Face shape: (67, 67, 3), x1=367, x2=434, y1=118, y2=185
Face shape: (134, 134, 3), x1=0, x2=134, y1=33, y2=167
Face shape: (96, 96, 3), x1=1273, x2=1369, y1=672, y2=768
Face shape: (57, 57, 3), x1=299, x2=356, y1=61, y2=118
Face shape: (459, 459, 3), x1=268, x2=727, y1=114, y2=573
Face shape: (79, 79, 3), x1=86, x2=165, y1=33, y2=112
Face shape: (192, 192, 3), x1=464, x2=656, y1=544, y2=736
Face shape: (193, 193, 3), x1=48, x2=241, y1=32, y2=225
Face shape: (96, 96, 3), x1=200, x2=296, y1=56, y2=152
Face shape: (164, 164, 3), x1=82, x2=246, y1=68, y2=232
Face shape: (95, 95, 3), x1=15, x2=110, y1=143, y2=238
Face shape: (0, 96, 3), x1=617, x2=713, y1=152, y2=248
Face shape: (157, 157, 3), x1=26, x2=183, y1=13, y2=170
Face shape: (95, 95, 3), x1=175, x2=270, y1=95, y2

#Data Collection and Counting:

 Displaying the count of processed data samples in the array X, which holds the resized facial images ready for training. This step provides an overview of the data size and quantity available for model training.

In [ ]:
print(len(X))

3916


#Converting Data to NumPy Arrays:

 Transforming the processed image data stored in the Python lists X and Y into NumPy arrays. This conversion is essential for compatibility with various machine learning frameworks and for further processing during model training and evaluation.

In [ ]:
import numpy as np
X = np.array(X)
Y = np.array(Y)

#Validating Data Conversion to NumPy Arrays:

 Checking whether the image data has been successfully converted to NumPy arrays. Depending on the conversion status, printing a message to indicate whether the conversion was successful or not. This step ensures the data is appropriately formatted for subsequent processing.

In [ ]:
if isinstance(X, np.ndarray) and isinstance(Y, np.ndarray):
    print("Images are converted to NumPy arrays.")
else:
    print("Images are not converted to NumPy arrays.")


Images are converted to NumPy arrays.


#Data Normalization and Shape Checking:

 Normalizing the pixel values of the image data in the array X to ensure they fall within the range [0, 1]. Additionally, checking and displaying the shape of the normalized data array X_normalized. This step prepares the data for optimal model training and highlights its structure.

In [ ]:
#X = X/255.0
X_normalized = X.astype('float32') / 255.0

# Check the shape of the array
print("Shape of X_normalized:", X_normalized.shape)

Shape of X_normalized: (3916, 64, 64, 3)


#Validation of Data Normalization Range:

 Verifying whether the data normalization process has been executed correctly by checking if all pixel values in the normalized data array X_normalized fall within the valid range [0, 1]. Based on the result of the check, printing a message indicating whether the images are properly normalized or providing information about the pixel value range if not. This step ensures that the data is appropriately scaled for optimal model training.

In [ ]:
if np.min(X_normalized) >= 0.0 and np.max(X_normalized) <= 1.0:
    print("Images are properly normalized.")
else:
    print("Images are not properly normalized. Pixel value range: [", np.min(X_normalized), ",", np.max(X_normalized), "]")

Images are properly normalized.


#Reshaping Images for Oversampling:

 Reshaping the image data in the array X to create a new representation that is suitable for oversampling techniques. The reshaped array X_new is organized as a flattened vector, which is a common format for input in oversampling algorithms. This step prepares the data for the subsequent oversampling process.





In [ ]:
#reshaping images for oversamlping
X_new = X.reshape(-1, 64*64*3)

#Applying Oversampling Using SMOTE:

 Employing the Synthetic Minority Over-sampling Technique (SMOTE) to oversample the image data in the flattened array X_new and its corresponding labels Y. The goal is to balance the class distribution by generating synthetic samples of the minority class. This step enhances the dataset's diversity and prevents class imbalance issues during model training.

In [ ]:
# oversampling the images

from imblearn.over_sampling import SMOTE

sampler = SMOTE()

XSMOTE, YSMOTE = sampler.fit_resample(X_new, Y)

#Analyzing Class Distribution After Oversampling:

 Computing and presenting the class distribution of the oversampled labels array YSMOTE using a Pandas Series. The value_counts() function tallies the occurrences of each class, providing insight into how the oversampling technique has affected the distribution of expression labels. This step verifies the effectiveness of the oversampling process in achieving class balance.

In [ ]:
pd.Series(YSMOTE).value_counts()

5    1648
6    1648
1    1648
4    1648
3    1648
0    1648
2    1648
dtype: int64

#Reshaping Images to Original Format:

 Reshaping the oversampled image data array XSMOTE back to its original format, where each image is represented as a 3D array with dimensions (height, width, channels). This transformation is necessary to ensure that the images are in the correct shape for input into convolutional neural networks (CNNs) and other image-based models.

In [ ]:
# again bringing images to its original shape
X = XSMOTE.reshape(-1, 64, 64, 3)

#Label Encoding and One-Hot Encoding:

 Converting the expression labels stored in the array Y into numerical format using label encoding. Subsequently, applying one-hot encoding to transform the integer-encoded labels into binary vectors. The shapes of the one-hot encoded labels are checked and printed to ensure the encoding process has been successful and to understand the structure of the encoded labels. This step prepares the labels for compatibility with classification models.

In [ ]:
#  label encoding, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded.reshape(-1, 1))

# Check the shape of onehot_encoded
print("Shape of onehot_encoded:", onehot_encoded.shape)

Shape of onehot_encoded: (3916, 7)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#Inspecting Encoded Labels:

 Displaying the first few rows of the one-hot encoded labels (onehot_encoded) to provide a visual understanding of how the labels have been transformed into binary vectors. This step helps in confirming the correctness of the encoding process and ensures that the labels are appropriately prepared for classification tasks.

In [ ]:
# Check the first few rows of onehot_encoded
print("First few rows of onehot_encoded:")
print(onehot_encoded[:5])

First few rows of onehot_encoded:
[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


#Image Data Augmentation Configuration:

 Configuring an instance of the ImageDataGenerator class from TensorFlow/Keras to perform data augmentation on the image data stored in the array X. Various augmentation techniques such as rescaling, rotation, shifting, shearing, zooming, and flipping are applied to artificially increase the diversity of the training data. The datagen.fit(X) line prepares the generator for augmenting the data during the training process. This step enhances the model's ability to generalize and perform well on new, unseen data.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True
)

datagen.fit(X)

#Building and Training Convolutional Neural Network (CNN) Model:

 This step involves constructing and training a CNN model using the Keras library. The process includes loading the CIFAR-10 dataset, preprocessing the data by normalizing pixel values and converting labels to one-hot encoded format. Data augmentation is applied using an ImageDataGenerator to enhance model generalization. The CNN architecture is built with convolutional layers, max-pooling layers, and dense layers. The model is compiled with an appropriate loss function and optimizer. It is then trained using the training data and evaluated on the test set to measure its performance in terms of loss and accuracy. This step is crucial for developing a model capable of recognizing facial expressions accurately.

In [ ]:
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize the pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Convert labels to one-hot encoded format
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Fit the ImageDataGenerator on the training data
datagen.fit(X_train)

# Define the batch size and number of epochs
batch_size = 32
epochs = 50

# Build the CNN model (using a simple architecture for demonstration purposes)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])

# Train the model with data augmentation and learning rate scheduling
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/50
1562/1562 [==============================] - 129s 82ms/step - loss: 1.7190 - accuracy: 0.3698 - val_loss: 1.3342 - val_accuracy: 0.5176
Epoch 2/50
1562/1562 [==============================] - 128s 82ms/step - loss: 1.3374 - accuracy: 0.5205 - val_loss: 1.1322 - val_accuracy: 0.5958
Epoch 3/50
1562/1562 [==============================] - 119s 76ms/step - loss: 1.2130 - accuracy: 0.5713 - val_loss: 1.0768 - val_accuracy: 0.6238
Epoch 4/50
1562/1562 [==============================] - 128s 82ms/step - loss: 1.1432 - accuracy: 0.5977 - val_loss: 1.0442 - val_accuracy: 0.6331
Epoch 5/50
1562/1562 [==============================] - 117s 75ms/step - loss: 1.0899 - accuracy: 0.6170 - val_loss: 0.9917 - val_accuracy: 0.6535
Epoch 6/50
1562/1562 [==============================] - 124s 80ms/step - loss: 1.0452 - accuracy: 0.6346 - val_loss: 0.9752 - val_accuracy: 0.6690
Epoch 7/50
1562/1562 [==============================] - 120s 77ms/step - loss: 1.0136 - accuracy: 0.6475 - val_loss: 0